Table 4.10.1: Injecting status of new presentations to treatment

In [1]:
from gssutils import *

if is_interactive():
    import requests
    from cachecontrol import CacheControl
    from cachecontrol.caches.file_cache import FileCache
    from cachecontrol.heuristics import LastModified
    from pathlib import Path

    session = CacheControl(requests.Session(),
                           cache=FileCache('.cache'),
                           heuristic=LastModified())

    sourceFolder = Path('in')
    sourceFolder.mkdir(exist_ok=True)

    inputURL = 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/752515/AdultSubstanceMisuseNDTMSDataTables2017-18.xlsx'
    inputFile = sourceFolder / 'AdultSubstanceMisuseNDTMSDataTables2017-18.xlsx'
    response = session.get(inputURL)
    with open(inputFile, 'wb') as f:
      f.write(response.content)    

In [2]:
tab = loadxlstabs(inputFile, sheetids='Table 4.10.1')[0]

Loading in\AdultSubstanceMisuseNDTMSDataTables2017-18.xlsx which has size 272149 bytes
Table names: ['Table 4.10.1']


https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/752515/AdultSubstanceMisuseNDTMSDataTables2017-18.xlsx

In [3]:
observations = tab.excel_ref('B5').expand(DOWN).expand(RIGHT).is_not_blank()
observations

{<F6 1407.0>, <C9 1.0>, <E9 1.0>, <F9 17893.0>, <J11 127307.0>, <H11 50656.0>, <I9 1.0>, <B6 14022.0>, <J6 17824.0>, <D9 15788.0>, <K7 0.0909958612724181>, <H7 108.0>, <B8 110.0>, <G8 0.00212373553903761>, <G7 0.0103392388084726>, <E5 0.905054471750697>, <I8 0.0023187015271448>, <D8 25.0>, <B9 40967.0>, <K6 0.142962558952806>, <D5 14289.0>, <I6 0.026045414567842>, <F11 18757.0>, <J8 289.0>, <E8 0.00158348112490499>, <G6 0.0786341027217348>, <H8 116.0>, <D7 382.0>, <I7 0.00215879107699688>, <J10 2631.0>, <K9 1.0>, <B10 211.0>, <H6 1303.0>, <J9 124676.0>, <B11 41178.0>, <H9 50028.0>, <D6 1092.0>, <B5 16165.0>, <B7 10670.0>, <F10 864.0>, <H5 48501.0>, <I5 0.969477092828016>, <G5 0.908902922930755>, <K8 0.00231800827745516>, <C8 0.00268508799765665>, <C6 0.34227549002856>, <D10 928.0>, <D11 16716.0>, <C5 0.394585886201089>, <J7 11345.0>, <F7 185.0>, <J5 95218.0>, <K5 0.763723571497321>, <C7 0.260453535772695>, <E7 0.0241955915885483>, <G9 1.0>, <F8 38.0>, <H10 628.0>, <E6 0.06916645553585>

In [4]:
status = tab.excel_ref('A5').expand(DOWN).is_not_blank() - tab.excel_ref('A11').expand(DOWN)
status

{<A7 'Currently injecting'>, <A8 'Declined to answer'>, <A9 'Total'>, <A10 'Missing/inconsistent'>, <A6 'Previously injected'>, <A5 'Never injected'>}

In [5]:
Clients = tab.excel_ref('B3').expand(RIGHT).is_not_blank()
Clients

{<D3 'Non-opiate only'>, <F3 'Non-opiate and Alcohol'>, <B3 'Opiate'>, <J3 'Total'>, <H3 'Alcohol only'>}

In [6]:
MeasureType = tab.excel_ref('B4').expand(RIGHT).is_not_blank()
MeasureType

{<J4 'n'>, <H4 'n'>, <C4 '%'>, <B4 'n'>, <E4 '%'>, <D4 'n'>, <I4 '%'>, <G4 '%'>, <F4 'n'>, <K4 '%'>}

In [7]:
Dimensions = [
            HDim(status,'Injecting Status',DIRECTLY,LEFT),
            HDim(Clients,'Clients in new treatment',CLOSEST,LEFT),
            HDim(MeasureType,'Measure Type',DIRECTLY,ABOVE),
            HDimConst('Unit','People')            
            ]

In [8]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
# if is_interactive():
#     savepreviewhtml(c1)

In [9]:
new_table = c1.topandas()
new_table

,OBS,Injecting Status,Clients in new treatment,Measure Type,Unit
0,16165.000000,Never injected,Opiate,n,People
1,0.394586,Never injected,Opiate,%,People
2,14289.000000,Never injected,Non-opiate only,n,People
3,0.905054,Never injected,Non-opiate only,%,People
4,16263.000000,Never injected,Non-opiate and Alcohol,n,People
5,0.908903,Never injected,Non-opiate and Alcohol,%,People
6,48501.000000,Never injected,Alcohol only,n,People
7,0.969477,Never injected,Alcohol only,%,People
8,95218.000000,Never injected,Total,n,People
9,0.763724,Never injected,Total,%,People


In [10]:
new_table = new_table[new_table['OBS'] != 0 ]

In [11]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [12]:
new_table.head()

,Value,Injecting Status,Clients in new treatment,Measure Type,Unit
0,16165.000000,Never injected,Opiate,n,People
1,0.394586,Never injected,Opiate,%,People
2,14289.000000,Never injected,Non-opiate only,n,People
3,0.905054,Never injected,Non-opiate only,%,People
4,16263.000000,Never injected,Non-opiate and Alcohol,n,People


In [13]:
new_table['Measure Type'] = new_table['Measure Type'].map(
    lambda x: {
        'n' : 'Count', 
        '%' : 'Percentage',
        }.get(x, x))

In [14]:
new_table.tail()

,Value,Injecting Status,Clients in new treatment,Measure Type,Unit
55,41178.0,None,Opiate,Count,People
56,16716.0,None,Non-opiate only,Count,People
57,18757.0,None,Non-opiate and Alcohol,Count,People
58,50656.0,None,Alcohol only,Count,People
59,127307.0,None,Total,Count,People


In [15]:
new_table.dtypes

Value                       float64
Injecting Status             object
Clients in new treatment     object
Measure Type                 object
Unit                         object
dtype: object

In [16]:
new_table['Value'] = new_table['Value'].astype(str)

In [17]:
new_table.head(3)

,Value,Injecting Status,Clients in new treatment,Measure Type,Unit
0,16165.0,Never injected,Opiate,Count,People
1,0.394585886201089,Never injected,Opiate,Percentage,People
2,14289.0,Never injected,Non-opiate only,Count,People


In [18]:
new_table['Clients in new treatment'] = new_table['Clients in new treatment'].map(
    lambda x: {
        'Total' : 'All Clients' 
        }.get(x, x))

In [19]:
new_table.head(3)

,Value,Injecting Status,Clients in new treatment,Measure Type,Unit
0,16165.0,Never injected,Opiate,Count,People
1,0.394585886201089,Never injected,Opiate,Percentage,People
2,14289.0,Never injected,Non-opiate only,Count,People


In [20]:
new_table['Injecting Status'].unique()

array(['Never injected', 'Previously injected', 'Currently injecting',
       'Declined to answer', 'Total', 'Missing/inconsistent', None],
      dtype=object)

In [21]:
new_table['Injecting Status'] = new_table['Injecting Status'].map(
    lambda x: {
        'Total' : 'All' 
        }.get(x, x))

In [22]:
def user_perc(x):
    
    if x == None:
        return 'All inclusice Inconsistent/missing'
    else:
        return x
    
new_table['Injecting Status'] = new_table.apply(lambda row: user_perc(row['Injecting Status']), axis = 1)

In [23]:
new_table = new_table[['Injecting Status','Clients in new treatment','Measure Type','Value','Unit']]

In [24]:
if is_interactive():
    SubstancetinationFolder = Path('out')
    SubstancetinationFolder.mkdir(exist_ok=True, parents=True)
    new_table.to_csv(SubstancetinationFolder / ('table4.10.1.csv'), index = False)